In [1]:
import json

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
from yandex.get_yandex_ratings import get_items

from sklearn.neighbors import KNeighborsClassifier

import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from matplotlib.markers import MarkerStyle
from postal.parser import parse_address
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.interpolate import spline
from sklearn.cluster import DBSCAN
from collections import Counter
import json



fontsize = 18
figsize = (15,10)
plot_alpha = 0.3
linewidth = 4

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [3]:
def str_to_point(s):
    if s is None:
        return None
    l = s.split(' ')
    return {'long': float(l[0]), 'lat': float(l[1])}

In [4]:
train = pd.read_csv('data/train.csv', index_col=0)
test = pd.read_csv('data/test.csv', index_col=0)
test = test.rename(columns={'atm_group': '0-bank'})
train = train.rename(columns={'atm_group': '0-bank'})
banks = np.uint(np.sort(test['0-bank'].unique() * 10))

with open('missed_coords.json', 'r') as infile:
    missed_coords = json.load(infile)

In [5]:
missed_coords = [(str_to_point(item[0]), item[1], item[2], item[3]) for item in missed_coords]

In [6]:
test = test.rename(columns={'lat': '6-lat', 'long': '6-long'})
train = train.rename(columns={'lat': '6-lat', 'long': '6-long'})

# Prepare

In [7]:
test['0-bank'] = np.uint(test['0-bank'] * 10)
train['0-bank'] = np.uint(train['0-bank'] * 10)
columns_for_drop = ['address', 'address_rus']

### Запоминаем индексы

In [8]:
train['is_train'] = True
test['is_train'] = False
test['index'] = test.index + 1
train['index'] = train.index + 1
train['train_index'] = train.index + 1
test['test_index'] = test.index + 1
train['test_index'] = 0
test['train_index'] = 0
train['index'] = train.index

### Создаём общие данные tran-test

In [9]:
data_all = train.append(test, sort=False)

data_all['address_fail'] = 0
data_all.at[data_all['address_rus'].isnull(), 'address_fail'] = 1
data_all['base_target'] = data_all['target']
data_all.at[(data_all['address_fail'] == 1) & (data_all['is_train']), 'target'] = 0

data_all['address_rus'] = data_all['address_rus'].fillna('')

counts = data_all.groupby('address_rus')['id'].count().reset_index().rename(columns={'id':'the_same_address_terminals'})
data_all = pd.merge(data_all, counts, how='left', on='address_rus')
data_all.at[data_all['address_fail'] == 1, 'the_same_address_terminals'] = 0

### Fillna for address_rus

In [10]:
data_all.at[data_all['address_fail'], 'address_rus'] = data_all['address_rus'].fillna("")

### Fillna for lat, long

In [11]:
data_all.at[:, ['6-lat', '6-long']] = data_all[['6-lat', '6-long']].fillna(0)

### Fill missed coords


In [12]:
missed_ids = [item[3] for item in missed_coords]
missed_cities = [item[2] for item in missed_coords]
missed_levels = [1 if item[1] == 'Level.HOUSE' else 0 for item in missed_coords]
missed_lats = [item[0]['lat'] if item[0] is not None else None for item in missed_coords]
missed_longs = [item[0]['long'] if item[0] is not None else None for item in missed_coords]

set_missed_ids = set(missed_ids)
missed_inds = data_all['id'].apply(lambda el: el in set_missed_ids)

In [13]:
data_all.at[missed_inds, '6-lat'] = missed_lats
data_all.at[missed_inds, '6-long'] = missed_longs
data_all['6-levels'] = 1
data_all.at[missed_inds, '6-levels'] = missed_levels
data_all.at[missed_inds, 'city'] = ['' if city is None else city.lower() for city in missed_cities]
data_all['address_fail'] = 0
data_all['address_fail'] = data_all['6-lat'].isnull()
data_all.at[:, ['6-lat', '6-long']] = data_all[['6-lat', '6-long']].fillna(0)

### Детекция городов в address_rus

In [14]:
def custom_parse_address(row):
    if len(row) == 0:
        return 'NULL'
    d = dict([(item[1], item[0]) for item in parse_address(row)])
    city = ''
    if 'city' in d:
        city = d['city']
    return city

data_all['city-temp'] = data_all['address_rus'].apply(custom_parse_address)

In [15]:
data_all.loc[missed_inds, 'city-temp'] = data_all[missed_inds]['city']
data_all['city'] = data_all['city-temp']
data_all['city'] = list(map(lambda x: x.replace('-', ' ').lower().replace('город ', '').replace(' город', '').replace('ё', 'e'), data_all['city']))
del data_all['city-temp']

In [16]:
# Не задетектились

d = {
    60: 'ст дружба-1',
    299: 'буденновский лес',
    300: 'буденновский лес',
    4443: 'екатеринбург',
    4444: 'екатеринбург',
    4445: 'екатеринбург',
    4446: 'екатеринбург',
    8374: 'усть-илимск',
    8386: 'усть-илимск',
    4609: 'усть-илимск'
}
for k in d:
    data_all.at[k, 'city'] = d[k]
data_all.at[data_all['city'] == '', 'city'] = data_all[data_all['city'] == ''].index.astype(str)

### Парсинг Яндекса

In [20]:
for i, addr in tqdm(enumerate(data_all['address_rus'].values), total=data_all.shape[0]):
    if i < 8553:
        continue
    resp = get_items('uohiuhiguyfytdtrcrytfugyug87t87t86r64ded6' if addr == '' else addr)
    with open('yandex/data/' + str(i) + '.json', 'w') as fd:
        json.dump(resp, fd)



  0%|          | 0/8765 [00:00<?, ?it/s]

 98%|█████████▊| 8554/8765 [00:00<00:00, 22869.54it/s]

 98%|█████████▊| 8554/8765 [00:13<00:00, 22869.54it/s]

 98%|█████████▊| 8580/8765 [00:14<00:29,  6.27it/s]   

 98%|█████████▊| 8581/8765 [00:14<00:48,  3.78it/s]

 98%|█████████▊| 8582/8765 [00:15<01:32,  1.99it/s]

 98%|█████████▊| 8583/8765 [00:16<01:52,  1.62it/s]

 98%|█████████▊| 8584/8765 [00:17<01:52,  1.60it/s]

 98%|█████████▊| 8585/8765 [00:18<02:23,  1.25it/s]

 98%|█████████▊| 8586/8765 [00:19<02:10,  1.37it/s]

 98%|█████████▊| 8587/8765 [00:19<01:53,  1.57it/s]

 98%|█████████▊| 8588/8765 [00:19<01:37,  1.81it/s]

 98%|█████████▊| 8589/8765 [00:20<01:43,  1.69it/s]

 98%|█████████▊| 8590/8765 [00:21<01:42,  1.71it/s]

 98%|█████████▊| 8591/8765 [00:21<01:40,  1.73it/s]

 98%|█████████▊| 8592/8765 [00:22<01:32,  1.88it/s]

 98%|█████████▊| 8593/8765 [00:22<01:33,  1.84it/s]

 98%|█████████▊| 8594/8765 [00:23<01:25,  1.99it/s]

 98%|█████████▊| 8595/8765 [00:23<01:32,  1.84

 99%|█████████▉| 8715/8765 [01:36<00:37,  1.32it/s]

 99%|█████████▉| 8716/8765 [01:37<00:36,  1.33it/s]

 99%|█████████▉| 8717/8765 [01:38<00:37,  1.29it/s]

 99%|█████████▉| 8718/8765 [01:39<00:36,  1.28it/s]

 99%|█████████▉| 8719/8765 [01:39<00:30,  1.52it/s]

 99%|█████████▉| 8720/8765 [01:40<00:34,  1.30it/s]

 99%|█████████▉| 8721/8765 [01:40<00:29,  1.48it/s]

100%|█████████▉| 8722/8765 [01:41<00:27,  1.59it/s]

100%|█████████▉| 8723/8765 [01:42<00:28,  1.46it/s]

100%|█████████▉| 8724/8765 [01:43<00:29,  1.41it/s]

100%|█████████▉| 8725/8765 [01:43<00:26,  1.53it/s]

100%|█████████▉| 8726/8765 [01:44<00:23,  1.64it/s]

100%|█████████▉| 8727/8765 [01:44<00:23,  1.61it/s]

100%|█████████▉| 8728/8765 [01:45<00:23,  1.56it/s]

100%|█████████▉| 8729/8765 [01:45<00:21,  1.65it/s]

100%|█████████▉| 8730/8765 [01:46<00:20,  1.70it/s]

100%|█████████▉| 8731/8765 [01:46<00:18,  1.82it/s]

100%|█████████▉| 8732/8765 [01:47<00:18,  1.75it/s]

100%|█████████▉| 8733/8765 [01:47<00:16,  1.95